In [1]:
%matplotlib inline

Traducción con una red de secuencia a secuencia y atención
==================




In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
print(torch.__version__)

2.0.1+cu118


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Carga de archivos de datos
==================



Representaremos cada palabra en un idioma como un vector de un solo calor, o un vector gigante de ceros, excepto uno solo (en el índice de la palabra). En comparación con las docenas de caracteres que pueden existir en un idioma, hay muchas más palabras, por lo que el vector de codificación es mucho más grande. Sin embargo, haremos trampa un poco y recortaremos los datos para usar solo unos pocos miles de palabras por idioma.





Necesitaremos un índice único por palabra para usar como entradas y objetivos de las redes más adelante. Para hacer un seguimiento de todo esto, usaremos una clase auxiliar llamada Lang que tiene diccionarios de índice de → de palabras (word2index) e índice → palabra (index2word), así como un recuento de cada palabra word2count para usar para luego reemplazar palabras raras.




In [5]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

Los archivos están todos en Unicode, para simplificar, convertiremos los caracteres Unicode a ASCII, haremos todo en minúsculas y recortaremos la mayoría de los signos de puntuación.




In [6]:
# Convierta una cadena Unicode en ASCII simple
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Minúsculas, recortar y quitar caracteres que no sean letras
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

Para leer el archivo de datos dividiremos el archivo en líneas, y luego dividiremos las líneas en pares. Los archivos son todos de inglés → otro idioma, así que si queremos traducir de Otro idioma → inglés, agregué la bandera reverse para invertir los pares.




In [7]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Lee el archivo y divídalo en líneas
    lines = open('/content/drive/MyDrive/IASeg/DATASETS/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Divide cada línea en pares y normaliza
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Pares inversos, crear instancias de Lang
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

Ya que hay un * montón * de oraciones de ejemplo y queremos entrenar
algo rápido, recortaremos el conjunto de datos a solo relativamente corto y
oraciones simples. Aquí la extensión máxima es de 10 palabras (que incluye
puntuación final) y estamos filtrando a oraciones que se traducen a
la forma "i am" o "he is", etc. (contabilizando los apóstrofos reemplazados
antes).




In [8]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

El proceso completo para preparar los datos es:

-  Leer archivo de texto y dividir en líneas, dividir líneas en pares
-  Normalizar texto, filtrar por longitud y contenido
-  Hacer listas de palabras a partir de oraciones en pares




In [9]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Tenemos %s pares de frases" % len(pairs))
    pairs = filterPairs(pairs)
    print("Recortando a %s pares de frases" % len(pairs))
    print("Conteo de palabras...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Palabras contadas:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'es', True)
print(random.choice(pairs))

Reading lines...
Tenemos 120614 pares de frases
Recortando a 7755 pares de frases
Conteo de palabras...
Palabras contadas:
es 4036
eng 2836
['estoy esperando una llamada .', 'i m waiting for a phone call .']


The Seq2Seq Model
=================

Una red neuronal recurrente, o RNN, es una red que opera en una secuencia y utiliza su propia salida como entrada para los pasos posteriores.

Una `red de secuencia a secuencia`__, o
red seq2seq, o `Encoder Decoder
red`__, es un modelo
que consta de dos RNN llamadas codificador y decodificador. El codificador lee
una secuencia de entrada y salidas de un solo vector, y el decodificador lee
ese vector para producir una secuencia de salida.


A diferencia de la predicción de secuencias con un solo RNN, donde cada entrada
Corresponde a una salida, el modelo seq2seq nos libera de la secuencia
longitud y orden, lo que lo hace ideal para la traducción entre dos
Idiomas.

Considere la frase "I'm not the black cat" → "Yo no soy el
gato negro". La mayoría de las palabras en la oración de entrada tienen un
traducción en la oración de salida, pero están en ligeramente diferente
órdenes, por ejemplo, "Chat Noir" y "Black Cat". Debido al "ne/pas"
Construcción También hay una palabra más en la oración de entrada. Sería
ser difícil producir una traducción correcta directamente de la secuencia
de palabras de entrada.

Con un modelo seq2seq el codificador crea un único vector que, en el
caso ideal, codifica el "significado" de la secuencia de entrada en un solo
vector — un solo punto en algún espacio de oraciones de N dimensiones.




Encoder
-----------

El codificador de una red seq2seq es un RNN que genera algún valor para cada palabra de la oración de entrada. Para cada palabra de entrada, el codificador genera un vector y un estado oculto, y utiliza el estado oculto para la siguiente palabra de entrada.




In [10]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size) #mapear
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

Decoder
-----------

El decodificador es otro RNN que toma los vectores de salida del codificador y genera una secuencia de palabras para crear la traducción.







En el decodificador seq2seq más simple usamos solo la última salida del codificador. Esta última salida a veces se denomina vector de contexto, ya que codifica el contexto de toda la secuencia. Este vector de contexto se utiliza como el estado oculto inicial del decodificador.

En cada paso de la decodificación, el decodificador recibe un token de entrada y un estado oculto. El token de entrada inicial es el token de inicio de cadena <SOS> , y el primer estado oculto es el vector de contexto (el último estado oculto del codificador).




In [11]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

Attention Decoder
========================
Si solo se pasa el vector de contexto entre el codificador y el decodificador, ese vector único lleva la carga de codificar toda la oración.

La atención permite que la red del decodificador se "enfoque" en una parte diferente de
Las salidas del codificador para cada paso de las propias salidas del decodificador. Primero
Calculamos un conjunto de *pesos de atención*. Estos se multiplicarán por
El codificador emite vectores para crear una combinación ponderada. El resultado
(denominado ''attn_applied'' en el código) debe contener información acerca de
esa parte específica de la secuencia de entrada, y así ayudar al decodificador
Elija las palabras de salida correctas.


El cálculo de los pesos de atención se realiza con otro feed-forward
capa ''attn'', utilizando la entrada del decodificador y el estado oculto como entradas.
Debido a que hay oraciones de todos los tamaños en los datos de entrenamiento, para
En realidad crear y entrenar esta capa tenemos que elegir un máximo
longitud de oración (longitud de entrada, para salidas de codificador) que puede aplicar
Para. Las oraciones de la longitud máxima usarán todos los pesos de atención,
mientras que las oraciones más cortas solo usarán las primeras.





In [12]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            F.relu(self.attn(torch.cat((embedded[0], hidden[0]), 1))), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

<div class="alert alert-info"><h4>Note</h4><p>There are other forms of attention that work around the length
  limitation by using a relative position approach. Read about "local
  attention" in `Effective Approaches to Attention-based Neural Machine
  Translation <https://arxiv.org/abs/1508.04025>`__.</p></div>

Training
========

Preparing Training Data
-----------------------

Para entrenar, para cada par necesitaremos un tensor de entrada (índices de las palabras en la oración de entrada) y un tensor objetivo (índices de las palabras en la oración objetivo). Al crear estos vectores, agregaremos el token EOS a ambas secuencias.




In [13]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

Training the Model
------------------

Para entrenar, ejecutamos la oración de entrada a través del codificador y realizamos un seguimiento de cada salida y el último estado oculto. A continuación, el decodificador recibe el token como su `<SOS>` primera entrada, y el último estado oculto del codificador como su primer estado oculto.


"Forzar a los maestros" es el concepto de utilizar los resultados objetivo reales como
cada entrada siguiente, en lugar de usar la suposición del decodificador como la siguiente entrada.
El uso de la fuerza del maestro hace que converja más rápido, pero 'cuando el entrenado
La red es explotada, puede exhibir
inestabilidad <http://minds.jacobs-university.de/sites/default/files/uploads/papers/ESNTutorialRev.pdf>`__.

Puede observar los resultados de las redes forzadas por el profesor que leen con
gramática coherente pero vagar lejos de la traducción correcta -
intuitivamente ha aprendido a representar la gramática de salida y puede "elegir
arriba" el significado una vez que el maestro le dice las primeras palabras, pero
no ha aprendido correctamente cómo crear la oración a partir de la traducción
en primer lugar.

Debido a la libertad que nos da el autograd de PyTorch, podemos
Elija usar Teacher Forzar o no con una simple declaración IF. Giro
``teacher_forcing_ratio`` hasta usar más.




In [14]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

Esta es una función auxiliar para imprimir el tiempo transcurrido y el tiempo restante estimado dado el tiempo actual y el porcentaje de progreso



In [15]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

Todo el proceso de entrenamiento se ve así:

-  Iniciar un temporizador
-  Inicializar optimizadores y criterios
-  Crear un conjunto de pares de entrenamiento
-  Iniciar matriz de pérdidas vacía para trazar

Luego llamamos al tren muchas veces y ocasionalmente imprimimos el progreso (% de ejemplos, tiempo hasta ahora, tiempo estimado) y la pérdida promedio.




In [16]:
checkpoint_iter = 5000  # Definir la frecuencia para guardar los checkpoints

In [17]:
import os


def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    checkpoint_file = '/content/drive/MyDrive/IASeg/CheckpointsS2S/checkpoint.pt'

    if os.path.exists(checkpoint_file):
        checkpoint = torch.load(checkpoint_file)  # Cargar el checkpoint previo

        encoder.load_state_dict(checkpoint['encoder_state_dict'])
        decoder.load_state_dict(checkpoint['decoder_state_dict'])
        encoder_optimizer.load_state_dict(checkpoint['encoder_optimizer_state_dict'])
        decoder_optimizer.load_state_dict(checkpoint['decoder_optimizer_state_dict'])
        iter = checkpoint['iter']
        loss = checkpoint['loss']
    else:
        checkpoint = None
        iter = 1
        loss = 0


    for iter in range(iter, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)

        if iter % checkpoint_iter == 0:
            if checkpoint is not None:  # Check if checkpoint is assigned
                torch.save(checkpoint, f'/content/drive/MyDrive/IASeg/CheckpointsS2S/checkpoint_{iter}.pt')

            checkpoint = {
                'encoder_state_dict': encoder.state_dict(),
                'decoder_state_dict': decoder.state_dict(),
                'encoder_optimizer_state_dict': encoder_optimizer.state_dict(),
                'decoder_optimizer_state_dict': decoder_optimizer.state_dict(),
                'loss': loss,
                'iter': iter,
                # Otras variables que desees guardar
            }
            # torch.save(checkpoint, f'checkpoint_{iter}.pt')

        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    if checkpoint is not None:  # Save the final checkpoint after the last iteration
        torch.save(checkpoint, f'/content/drive/MyDrive/IASeg/CheckpointsS2S/checkpoint.pt')

    showPlot(plot_losses)

Trazar resultados
----------------

El trazado se realiza con matplotlib, utilizando la matriz de valores de pérdida
''plot_losses'' guardado durante el entrenamiento.




In [18]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

Evaluación
==========

La evaluación es casi lo mismo que la capacitación, pero no hay objetivos, por lo que
Simplemente alimentamos las predicciones del decodificador a sí mismo para cada paso.
Cada vez que predice una palabra la agregamos a la cadena de salida, y si
predice el token EOS que detenemos allí. También almacenamos los decodificadores
Salidas de atención para su visualización posterior.


In [19]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

Podemos evaluar oraciones aleatorias del conjunto de entrenamiento e imprimir la entrada, el objetivo y la salida para hacer algunos juicios de calidad subjetivos:

In [20]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

Entrenamiento y evaluación
=======================

Con todas estas funciones auxiliares en su lugar (parece trabajo extra, pero
hace que sea más fácil ejecutar múltiples experimentos) en realidad podemos
Inicialice una red y comience a entrenar.

Recuerde que las oraciones de entrada estaban muy filtradas. Para este pequeño
conjunto de datos podemos utilizar redes relativamente pequeñas de 256 nodos ocultos y un
una sola capa GRU. Después de unos 40 minutos en una CPU de MacBook obtendremos algunos
resultados razonables.


.. Nota::
   Si ejecuta este cuaderno puede entrenar, interrumpir el kernel,
   Evalúe y continúe la capacitación más tarde. Comente las líneas donde el
   el codificador y el decodificador se inicializan y vuelven a ejecutar ''trainIters''.



In [21]:
device = "cuda"

In [22]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

1m 33s (- 21m 43s) (5000 6%) 3.1290
3m 6s (- 20m 12s) (10000 13%) 2.4831
4m 37s (- 18m 28s) (15000 20%) 2.0803
6m 6s (- 16m 47s) (20000 26%) 1.7976
7m 38s (- 15m 16s) (25000 33%) 1.5395
9m 8s (- 13m 43s) (30000 40%) 1.3148
10m 40s (- 12m 12s) (35000 46%) 1.1817
12m 11s (- 10m 40s) (40000 53%) 1.0296
13m 42s (- 9m 8s) (45000 60%) 0.8927
15m 14s (- 7m 37s) (50000 66%) 0.8194
16m 45s (- 6m 5s) (55000 73%) 0.6952
18m 30s (- 4m 37s) (60000 80%) 0.6463
20m 4s (- 3m 5s) (65000 86%) 0.5575
21m 36s (- 1m 32s) (70000 93%) 0.4987
23m 7s (- 0m 0s) (75000 100%) 0.4501


In [23]:
evaluateRandomly(encoder1, attn_decoder1)

> juega bien al tenis .
= he is good at playing tennis .
< he is good at tennis . <EOS>

> estoy muy avergonzado .
= i m too ashamed .
< i m very embarrassed . <EOS>

> estoy fascinada .
= i m fascinated .
< i m fascinated . <EOS>

> no soy bueno para el frances .
= i m not good at french .
< i m not good at french . <EOS>

> el es un hombre comun .
= he s just an ordinary man .
< he s just an man man . <EOS>

> no sabes nadar verdad ?
= you are not able to swim are you ?
< you are not able to to you ? <EOS>

> soy de china .
= i am from china .
< i am from china . <EOS>

> soy la mas alta de nuestra clase .
= i am the tallest in our class .
< i am the tallest in our class . <EOS>

> tiene cincuenta y tantos anos .
= he s in his fifties .
< he is thirty two . <EOS>

> me interesa el alpinismo .
= i am interested in mountain climbing .
< i am interested in mountain climbing . <EOS>



Visualizando la atención
---------------------

Una propiedad útil del mecanismo de atención es su alta interpretabilidad
Salidas. Porque se utiliza para ponderar salidas de codificador específicas del
secuencia de entrada, podemos imaginar mirando hacia dónde se enfoca más la red
en cada paso de tiempo.

Simplemente puede ejecutar ''plt.matshow (atenciones)'' para ver la salida de atención
se muestra como una matriz, con las columnas siendo pasos de entrada y filas siendo
Pasos de salida:




In [24]:
output_words, attentions = evaluate(
    encoder1, attn_decoder1, "tengo demasiado frio .")
plt.matshow(attentions.numpy())
output_words

['i', 'am', 'very', 'cold', '.', '<EOS>']

Para una mejor experiencia de visualización haremos el trabajo extra de añadir ejes y etiquetas:



In [25]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(
        encoder1, attn_decoder1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions)


evaluateAndShowAttention("ella es cinco anos mas joven que yo .")

evaluateAndShowAttention("es demasiado pequeno .")

evaluateAndShowAttention("no temo morir .")

evaluateAndShowAttention("es un joven director talentoso .")

input = ella es cinco anos mas joven que yo .
output = she s five years older than i am . <EOS>
input = es demasiado pequeno .
output = he s too dangerous . <EOS>
input = no temo morir .
output = i m not able . <EOS>
input = es un joven director talentoso .
output = he s a very good man . <EOS>


<ipython-input-25-8b5881de891c>:9: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([''] + input_sentence.split(' ') +
<ipython-input-25-8b5881de891c>:11: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels([''] + output_words)


In [26]:
!pip3 install pytorch-pretrained-bert

In [27]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForSequenceClassification

# Cargar tokenizador de modelo pre-entrenado (vocabulario)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Entrada tokenizada
text = "Who was Jim Henson ? Jim Henson was a puppeteer"
tokenized_text = tokenizer.tokenize(text)

# Enmascara un token que intentaremos predecir con 'BertForMaskedLM'
masked_index = 6
tokenized_text[masked_index] = '[MASK]'
assert tokenized_text == ['who', 'was', 'jim', 'henson', '?', 'jim', '[MASK]', 'was', 'a', 'puppet', '##eer']

# Convertir token en índices de vocabulario
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# Definir los índices de las frases A y B asociados a la 1ª y 2ª oración (ver artículo)
segments_ids = [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

In [28]:
# Modelo pre-entrenado de carga (pesos)
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

# Predecir entidades de estados ocultos para cada capa
encoded_layers, _ = model(tokens_tensor, segments_tensors)

# Tenemos un estado oculto para cada una de las 12 capas en el modelo bert-base-uncased
assert len(encoded_layers) == 12

In [29]:
# Modelo pre-entrenado de carga (pesos)
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()

# Predecir todos los tokens
predictions = model(tokens_tensor, segments_tensors)

# confirmar que pudimos predecir 'Henson'
predicted_index = torch.argmax(predictions[0, masked_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
assert predicted_token == 'henson'

In [30]:
print(encoded_layers[1].size())

torch.Size([1, 11, 768])


In [31]:
print(_.size())

torch.Size([1, 768])


In [32]:
def swish(x):
    return x * torch.sigmoid(x)

class BERTEncoder(BertForSequenceClassification):
    def __init__(self, config, num_labels=2):
        super(BERTEncoder, self).__init__(config, num_labels=num_labels)
        self.num_labels = num_labels
        self.bert = BertModel(config) #capa codificador de BERT
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, num_labels)
        self.apply(self.init_bert_weights)



    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        _, pooled_output = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        logits = swish(logits)

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            return loss
        else:
            return logits




In [33]:
model = BERTEncoder.from_pretrained('bert-base-uncased')
model.eval()

# Predict all tokens
predictions = model(tokens_tensor)

In [34]:
print(predictions.size())

torch.Size([1, 2])


In [40]:
device = "cuda"

In [41]:
# hidden_size = 256
# enc = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=hidden_size).to(device)
# for param in enc.bert.parameters():
#     param.requires_grad = False
# decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)

In [42]:
teacher_forcing_ratio = 0.5


def trainBert(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    #encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    encoder_hidden = encoder(input_tensor)

    encoder_hidden = encoder_hidden.unsqueeze(0)

    #for ei in range(input_length):
        #encoder_output, encoder_hidden = encoder(
            #input_tensor[ei], encoder_hidden)
        #encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [43]:
checkpoint_iter = 5000  # Definir la frecuencia para guardar los checkpoints

In [44]:
import os

def trainItersBert(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01, mom=0):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate, momentum=mom)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate, momentum=mom)

    #encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate, amsgrad=True)
    #encoder_scheduler = optim.lr_scheduler.CosineAnnealingLR(encoder_optimizer, n_iters)
    #decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate, amsgrad=True)
    #decoder_scheduler = optim.lr_scheduler.CosineAnnealingLR(decoder_optimizer, n_iters)

    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    checkpoint_file = '/content/drive/MyDrive/IASeg/CheckpointsBERT/checkpoint.pt'

    if os.path.exists(checkpoint_file):
        checkpoint = torch.load(checkpoint_file)  # Cargar el checkpoint previo

        encoder.load_state_dict(checkpoint['encoder_state_dict'])
        decoder.load_state_dict(checkpoint['decoder_state_dict'])
        encoder_optimizer.load_state_dict(checkpoint['encoder_optimizer_state_dict'])
        decoder_optimizer.load_state_dict(checkpoint['decoder_optimizer_state_dict'])
        iter = checkpoint['iter']
        loss = checkpoint['loss']
        print(iter)
    else:
        checkpoint = None
        iter = 1
        loss = 0

    for iter in range(iter, n_iters + 1):
        #encoder_scheduler.step()
        #decoder_scheduler.step()

        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]
        input_tensor.transpose_(0,1)
        #print(input_tensor.size())

        loss = trainBert(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)

        if iter % checkpoint_iter == 0:
            if checkpoint is not None:  # Check if checkpoint is assigned
                torch.save(checkpoint, f'/content/drive/MyDrive/IASeg/CheckpointsBERT/checkpoint_{iter}.pt')

            checkpoint = {
                'encoder_state_dict': encoder.state_dict(),
                'decoder_state_dict': decoder.state_dict(),
                'encoder_optimizer_state_dict': encoder_optimizer.state_dict(),
                'decoder_optimizer_state_dict': decoder_optimizer.state_dict(),
                'loss': loss,
                'iter': iter,
                # Otras variables que desees guardar
            }

        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0


    if checkpoint is not None:  # Save the final checkpoint after the last iteration
        torch.save(checkpoint, f'/content/drive/MyDrive/IASeg/CheckpointsBERT/checkpoint.pt')

    showPlot(plot_losses)

In [46]:
seq = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=hidden_size)

In [47]:
out = seq(tokens_tensor)

In [48]:
print(out.size())

torch.Size([1, 256])


In [50]:
hidden_size = 256
decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)

In [51]:
enc2 = BERTEncoder.from_pretrained('bert-base-multilingual-cased', num_labels=hidden_size).to(device)
for param in enc2.bert.parameters():
    param.requires_grad = False

In [52]:
trainItersBert(enc2, decoder1, 75000, print_every=500, learning_rate=0.01, mom=0.001)

0m 19s (- 48m 46s) (500 0%) 3.7123
0m 36s (- 44m 40s) (1000 1%) 3.4998
0m 49s (- 40m 28s) (1500 2%) 3.4420
1m 2s (- 38m 15s) (2000 2%) 3.3773
1m 16s (- 37m 4s) (2500 3%) 3.3066
1m 29s (- 35m 58s) (3000 4%) 3.2453
1m 43s (- 35m 9s) (3500 4%) 3.1833
1m 58s (- 34m 56s) (4000 5%) 3.2159
2m 11s (- 34m 22s) (4500 6%) 3.2792
2m 25s (- 33m 52s) (5000 6%) 3.1936
2m 40s (- 33m 42s) (5500 7%) 3.1970
2m 53s (- 33m 14s) (6000 8%) 3.1470
3m 6s (- 32m 47s) (6500 8%) 3.1756
3m 20s (- 32m 24s) (7000 9%) 3.1036
3m 33s (- 32m 4s) (7500 10%) 3.0973
3m 47s (- 31m 44s) (8000 10%) 3.0809
4m 9s (- 32m 28s) (8500 11%) 3.0854
4m 22s (- 32m 6s) (9000 12%) 3.1048
4m 36s (- 31m 46s) (9500 12%) 3.0044
4m 54s (- 31m 55s) (10000 13%) 3.0718
5m 9s (- 31m 42s) (10500 14%) 3.0231
5m 24s (- 31m 29s) (11000 14%) 3.1279
5m 38s (- 31m 8s) (11500 15%) 3.0840
5m 52s (- 30m 48s) (12000 16%) 3.0812
6m 6s (- 30m 33s) (12500 16%) 2.9942
6m 20s (- 30m 13s) (13000 17%) 3.0361
6m 33s (- 29m 54s) (13500 18%) 3.0740
6m 47s (- 29m 36s)

In [53]:
def evaluate(encoder, decoder, sentence, tokenizer, max_length=10):
    with torch.no_grad():
        input_tokens = tokenizer.tokenize(sentence)
        input_indices = tokenizer.convert_tokens_to_ids(input_tokens)
        input_tensor = torch.tensor([input_indices], device=device)
        input_length = input_tensor.size(1)
        attention_mask = torch.tensor([[1] * input_length], device=device)

        encoder_hidden = encoder(input_tensor, attention_mask=attention_mask)

        decoder_input = torch.tensor([[SOS_token]], device=device)
        decoder_hidden = encoder_hidden.unsqueeze(0)  # Agregar dimensión adiciona

        decoded_words = []

        for _ in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [54]:
output_words= evaluate(
    enc2, decoder1, "no tengo miedo a morir .",tokenizer)
# plt.matshow(attentions.numpy())
translation = ' '.join(output_words)
translation

'i m not . . <EOS>'